# IMDB Review Sentiment Analysis
This should explain the problem with some links

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import os
import sys

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [2]:
# Checking if we have the data. If not download it
# This needs to be done...
# Mention that we could use the built-in keras.load_imdb() but want to do it manually for clarity
VOCAB_PATH = 'data/imdb.vocab'
TRAIN_PATH = 'data/train'
TEST_PATH = 'data/test'

SEED = 2018
VOCAB_SIZE = 100
MAX_REVIEW_LEN = 250

In [3]:
def get_x_y(file_path):
    files = {}
    files['pos'] = glob(os.path.join(file_path, 'pos', '*.txt'))
    files['neg'] = glob(os.path.join(file_path, 'neg', '*.txt'))
    
    sentiment_map = {'pos': 1, 'neg': 0}
    x = []
    y = []
    for sentiment in files:
        for file_name in files[sentiment]:
            temp_ = []
            with open(file_name) as file_:
                temp_ = file_.read()
            x.append(temp_)
            y.append(sentiment_map[sentiment])
            
    return x, y

In [4]:
# Read in the text data
x_train, y_train = get_x_y(TRAIN_PATH)
x_test, y_test = get_x_y(TEST_PATH)

In [5]:
# Make our tokenizer
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(x_train)

In [6]:
# Fit our training data
x_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(x_train, maxlen=MAX_REVIEW_LEN)

# Fit our testing data
x_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(x_test, maxlen=MAX_REVIEW_LEN)

In [7]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(VOCAB_SIZE, embedding_vecor_length, input_length=MAX_REVIEW_LEN))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 32)           3200      
_________________________________________________________________
dropout_1 (Dropout)          (None, 250, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 56,501
Trainable params: 56,501
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
model.fit(x_train, y_train, epochs=3, batch_size=64)

Epoch 1/3
25000/25000 [==============================] - 50s 2ms/step - loss: 0.6043 - acc: 0.6646
Epoch 2/3
25000/25000 [==============================] - 52s 2ms/step - loss: 0.5547 - acc: 0.7229
Epoch 3/3
25000/25000 [==============================] - 51s 2ms/step - loss: 0.5428 - acc: 0.7286


In [10]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 73.33%
